## Title :

Exercise: Random Forest with Class Imbalance

## Description :

The goal of this exercise is to investigate the performance of Random Forest on a dataset with class imbalance, and then use corrections strategies to improve performance.

Your final comparison may look like the table below (but not with these exact values):

<img src="../fig/fig1.png" style="width: 500px;">

## Instructions:

- Read the dataset `diabetes.csv` as a pandas dataframe.
- Take a quick look at the dataset.
- Split the data into train and test sets.
- Perform classification with a Vanilla Random Forest which does not take into account class imbalance.
- Perform classification with a Balanced Random Forest which does take into account class imbalance.
- Upsample the data and perform classification with a Balanced Random Forest.
- Downsample the data and perform classification with a Balanced Random Forest.
- Compare the F1-Score and AUC Score of all 4 models.

## Hints: 

<a href="https://numpy.org/doc/stable/reference/generated/numpy.ravel.html" target="_blank">np.ravel()</a>
Return a contiguous flattened array.

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html" target="_blank">f1_score()</a>
Compute the F1 score, also known as balanced F-score or F-measure.

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html" target="_blank">roc_auc_score()</a>
Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html" target="_blank">sklearn.train_test_split()</a>
Split arrays or matrices into random train and test subsets.

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html" target="_blank">RandomForestClassifier()</a>
Defines the RandomForestClassifier and includes more details on the definition and range of values for its tunable parameters.

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.fit" target="_blank">RandomForestClassifier.fit()</a>
Build a forest of trees from the training set (X, y).

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.predict" target="_blank">RandomForestClassifier.predict()</a>
Predict class for X.

<a href="https://imbalanced-learn.org/stable/references/generated/imblearn.ensemble.BalancedRandomForestClassifier.html" target="_blank">BalancedRandomForestClassifier()</a>
A balanced random forest classifier.

<a href="https://imbalanced-learn.org/stable/references/generated/imblearn.ensemble.BalancedRandomForestClassifier.html#imblearn.ensemble.BalancedRandomForestClassifier.fit" target="_blank">BalancedRandomForestClassifier.fit()</a>
Build a forest of trees from the training set (X, y).

<a href="https://imbalanced-learn.org/stable/references/generated/imblearn.ensemble.BalancedRandomForestClassifier.html#imblearn.ensemble.BalancedRandomForestClassifier.predict" target="_blank">BalancedRandomForestClassifier.predict()</a>
Predict class for X.

<a href="https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html" target="_blank">SMOTE()</a>
Class to perform over-sampling using SMOTE.

<a href="https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html#imblearn.over_sampling.SMOTE.fit_resample" target="_blank">SMOTE.fit_resample()</a>
Resample the dataset.

<a href="https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.RandomUnderSampler.html" target="_blank">RandomUnderSampler()</a>
Class to perform random under-sampling.

<a href="https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.RandomUnderSampler.html#imblearn.under_sampling.RandomUnderSampler.fit_resample" target="_blank">RandomUnderSampler.fit_resample()</a>
Resample the dataset.

In [50]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from imblearn.under_sampling import RandomUnderSampler
from imblearn.ensemble import BalancedRandomForestClassifier
%matplotlib inline


In [51]:
# Code to read the dataset and take a quick look
df = pd.read_csv("../data/diabetes.csv")
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [52]:
# Investigate the response variable for data imbalance
count0, count1 = df['Outcome'].value_counts()
# f['Outcome'].value_counts() returns a Series, count0 and count1 get the Series's values

print(f'The percentage of diabetics in the dataset is only {100*count1/(count0+count1):.2f}%')


The percentage of diabetics in the dataset is only 34.90%


In [53]:
# Assign the predictor and response variables
# "Outcome" is the response and all the other columns are the predictors
# Use the values of these features and response
X = df.drop(columns=["Outcome"])
y = df["Outcome"]


In [54]:
# Fix a random_state
random_state = 22

# Split the data into train and validation sets
# Set random state as defined above and use a train size of 0.8
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=random_state)


In [55]:
# Set the max_depth variable to 20 for all trees
max_depth = 20


## Strategy 1 - Vanilla Random Forest

- No correction for imbalance

In [56]:
# Define a Random Forest classifier with randon_state as above
# Set the maximum depth to be max_depth and use 10 estimators
random_forest = RandomForestClassifier(n_estimators=10, random_state=random_state, max_depth=max_depth)

# Fit the model on the training set
random_forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=20, n_estimators=10, random_state=22)

In [57]:
### edTest(test_vanilla) ### 
# Use the trained model to predict on the validation set 
predictions = random_forest.predict(X_val)

# Compute two metrics that better represent misclassification of minority classes 
# i.e `F1 score` and `AUC`

# Compute the F1-score and assign it to variable score1
f_score = f1_score(y_val, predictions)
score1 = round(f_score, 2)

# Compute the AUC and assign it to variable auc1
auc_score = roc_auc_score(y_val, predictions)
auc1 = round(auc_score, 2)

auc1_= auc4_ = round(roc_auc_score(y_val,random_forest.predict_proba(X_val)[:, 1]),2)


## Strategy 2 - Random Forest with class weighting
- Balancing the class imbalance in each bootstrap

In [58]:
# Define a Random Forest classifier with randon_state as above
# Set the maximum depth to be max_depth and use 10 estimators
# Use class_weight as balanced_subsample to weigh the class accordingly
random_forest = RandomForestClassifier(n_estimators=10, random_state=random_state, max_depth=max_depth, class_weight="balanced_subsample")

# Fit the model on the training set
random_forest.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced_subsample', max_depth=20,
                       n_estimators=10, random_state=22)

In [59]:
### edTest(test_balanced) ###

# Use the trained model to predict on the validation set 
predictions = random_forest.predict(X_val)

# Compute two metrics that better represent misclassification of minority classes 
# i.e `F1 score` and `AUC`

# Compute the F1-score and assign it to variable score2
f_score = f1_score(y_val, predictions)
score2 = round(f_score, 2)

# Compute the AUC and assign it to variable auc2
auc_score = roc_auc_score(y_val, predictions)
auc2 = round(auc_score, 2)
auc2_ = round(roc_auc_score(y_val,random_forest.predict_proba(X_val)[:, 1]),2)

## Strategy 3 - Balanced Random Forest with SMOTE 

- Using the **imblearn** `BalancedRandomForestClassifier()` 
- Read more about this implementation [here](https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.ensemble.BalancedRandomForestClassifier.html)

In [60]:
# Perform upsampling using SMOTE

# Define a SMOTE with random_state=2
sm = SMOTE(random_state=2)

# Use the SMOTE object to upsample the train data
# You may have to use ravel() 
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)


In [61]:
# Define a Random Forest classifier with randon_state as above
# Set the maximum depth to be max_depth and use 10 estimators
# Use class_weight as balanced_subsample to weigh the class accordingly
random_forest = RandomForestClassifier(n_estimators=10, random_state=random_state, max_depth=max_depth, class_weight="balanced_subsample")

# Fit the Random Forest on upsampled data 
random_forest.fit(X_train_res, y_train_res)


RandomForestClassifier(class_weight='balanced_subsample', max_depth=20,
                       n_estimators=10, random_state=22)

In [62]:
### edTest(test_upsample) ###

# Use the trained model to predict on the validation set 
predictions = random_forest.predict(X_val)

# Compute the F1-score and assign it to variable score3
f_score = f1_score(y_val, predictions)
score3 = round(f_score, 2)

# Compute the AUC and assign it to variable auc3
auc_score = roc_auc_score(y_val, predictions)
auc3 = round(auc_score, 2)
auc3_ = round(roc_auc_score(y_val,random_forest.predict_proba(X_val)[:, 1]),2)


## Strategy 4 - Downsample the data

Using the imblearn RandomUnderSampler().

In [63]:
# Define an RandomUnderSampler instance with random state as 2
rs = RandomUnderSampler(random_state=2)

# Downsample the train data
# You may have to use ravel()
X_train_res, y_train_res = rs.fit_resample(X_train, y_train)


In [64]:
# Define a Random Forest classifier with randon_state as above
# Set the maximum depth to be max_depth and use 10 estimators
# Use class_weight as balanced_subsample to weigh the class accordingly
random_forest = RandomForestClassifier(n_estimators=10, random_state=random_state, max_depth=max_depth, class_weight="balanced_subsample")

# Fit the Random Forest on downsampled data 
random_forest.fit(X_train_res, y_train_res)


RandomForestClassifier(class_weight='balanced_subsample', max_depth=20,
                       n_estimators=10, random_state=22)

In [65]:
### edTest(test_downsample) ###

# Use the trained model to predict on the validation set 
predictions = random_forest.predict(X_val)

# Compute two metrics that better represent misclassification of minority classes 
# i.e `F1 score` and `AUC`

# Compute the F1-score and assign it to variable score4
f_score = f1_score(y_val, predictions)
score4 = round(f_score, 2)

# Compute the AUC and assign it to variable auc4
auc_score = roc_auc_score(y_val, predictions)
auc4 = round(auc_score, 2)

auc4_ = round(roc_auc_score(y_val,random_forest.predict_proba(X_val)[:, 1]),2)


In [66]:
# Compile the results from the implementations above

pt = PrettyTable()
pt.field_names = ["Strategy","F1 Score","AUC score", "AUC score_"]
pt.add_row(["Random Forest - No imbalance correction",score1,auc1, auc1_])
pt.add_row(["Random Forest - balanced_subsamples",score2,auc2, auc2_])
pt.add_row(["Random Forest - Upsampling",score3,auc3, auc3_])
pt.add_row(["Random Forest - Downsampling",score4,auc4, auc4_])
print(pt)


+-----------------------------------------+----------+-----------+------------+
|                 Strategy                | F1 Score | AUC score | AUC score_ |
+-----------------------------------------+----------+-----------+------------+
| Random Forest - No imbalance correction |   0.44   |    0.62   |    0.75    |
|   Random Forest - balanced_subsamples   |   0.45   |    0.62   |    0.73    |
|        Random Forest - Upsampling       |   0.54   |    0.66   |    0.74    |
|       Random Forest - Downsampling      |   0.64   |    0.72   |    0.77    |
+-----------------------------------------+----------+-----------+------------+


### ⏸ Which of the metrics given below is not recommended when there is a imbalance in the dataset?

#### A. Precision
#### B. Recall
#### C. F1-Score
#### D. Accuracy
#### E. AUC-Score

In [67]:
### edTest(test_chow1) ###
# Submit an answer choice as a string below (eg. if you choose option A, put 'A')
answer1 = '___'
